<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

<!--- @wandbcode{artifacts-fundamentals} -->


# W&B Workshop: Aquatic Species Classification

### From First Baseline to Production-Ready Model with Full MLOps

---

### Premise

**Scenario:** You're part of a marine biology AI research team building an image classifier to identify aquatic species from underwater photographs. The goal is to help marine researchers automatically catalog and monitor marine biodiversity.

**Your journey:**
1. **Train a baseline model** with experiment tracking and visual diagnostics
2. **Package the model** as a versioned artifact with lineage back to the training data
3. **Stage the baseline** in the Model Registry, then **sweep** for better hyperparameters
4. **Promote the winner** to production based on sweep results

---

### What You'll Learn

| Section | Topic | Key Skills |
|---------|-------|------------|
| 1 | Setup | Environment configuration, W&B login, config objects |
| 2 | Data & Artifacts | Consume pre-prepared dataset artifacts, lineage with use_artifact |
| 3 | Experiment Tracking | Run initialization, logging profiles, step alignment, commit=False |
| 4 | Model Training | PyTorch training, mixed precision, checkpoint logging, TTL |
| 5 | Visual Logging | Images, tables, ROC curves, per-class metrics |
| 6 | Model Artifacts | Model artifact with lineage, reference artifacts, use_artifact |
| 7 | Registry | Collections, linking, staging the baseline |
| 8 | Sweeps | Hyperparameter optimization, sweep vs baseline, promote winner |
| 9 | Offline Mode | WANDB_MODE, syncing runs |
| 10 | Programmatic Reports | Reports API, blocks, PanelGrid, automated documentation |
| 11 | Wrap-up | Recap, next steps |

## 1. Setup

Let's install dependencies, authenticate with W&B, and configure our experiment.

The `workshop_utils.py` file handles all ML boilerplate (transforms, dataset class, etc.).

In [ ]:
# If you haven't already installed dependencies:
# !pip install -r ../requirements.txt -q

In [ ]:
# Imports
import random
import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler
import wandb
from sklearn.metrics import precision_recall_fscore_support
from datetime import datetime, timedelta
import json
import os

# Workshop utilities (handles ML boilerplate)
from workshop_utils import (
    CLASS_NAMES, NUM_CLASSES, DEVICE,
    set_seed, get_transforms,
    create_model, count_parameters,
    train_one_epoch, evaluate,
    generate_run_name,
    AquaticDataset,
    create_dataloaders, create_training_components,
    save_checkpoint, log_checkpoint_artifact,
    create_prediction_images, create_predictions_table,
)

print(f"PyTorch: {torch.__version__}")
print(f"Device: {DEVICE}")
print(f"W&B: {wandb.__version__}")


## 🪄 Log in to W&B
- You can explicitly login using `wandb login` or `wandb.login()` (See below)
- Alternatively you can set [optional environment variables](https://docs.wandb.ai/guides/track/environment-variables/#optional-environment-variables). There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - `WANDB_API_KEY` - find this in your "Settings" section under your profile
    - `WANDB_BASE_URL` - this is the url of the W&B server
- Find your API Token in "Prof

In [ ]:
WANDB_ENTITY = "m-bakir"     # W&B team name
WANDB_PROJECT = "SIE-Workshop-2026"    # Project name

# Authenticate with W&B
WANDB_HOST = "https://mbakir.wandb.io/" 
wandb.login(host= WANDB_HOST)

# app. This has been changed to your private instance
# WANDB_HOST = "<https://<user-defined-full-url>" 

# Alternative you can configure this with environment variables:
# export WANDB_API_KEY="<your-api-key>"
# export WANDB_BASE_URL="<your-wandb-endpoint>"

## Configuration

Single config object for all hyperparameters and run metadata. You will be assigned a random open source model.

In [ ]:
# Random model assignment - creates diverse runs for W&B comparison!
WORKSHOP_MODELS = ["resnet50", "efficientnet_b0"]
ASSIGNED_MODEL = random.choice(WORKSHOP_MODELS)
print(f"Your assigned model: {ASSIGNED_MODEL}")

# Training config - these get logged to W&B automatically
CONFIG = {
    "model_name": ASSIGNED_MODEL,
    "num_classes": NUM_CLASSES,
    "epochs": 3,              # Quick training for workshop
    "batch_size": 32,
    "learning_rate": 1e-3,
    "weight_decay": 1e-4,
    "image_size": 224,
    "max_samples": 1000,      # Subset for fast iteration
    "use_amp": True,
    "seed": 42,
}

# Set reproducibility
set_seed(CONFIG["seed"])
print(f"\nConfig: {CONFIG}")
print(f"Using device: {DEVICE}")

## 2. Data Preparation

The AQUA dataset has been **pre-prepared, logged as W&B artifacts, and pre-loaded into your local environment**. This mirrors a common production pattern where data lives on shared storage (NFS, S3, a team drive) and teams use W&B to **track and version** it without re-downloading every run.

**What's in your local `data/` directory:**
- `data/train/` - Training split (~6,500 images, 20 class subfolders)
- `data/val/` - Validation split (~800 images)  
- `data/test/` - Test split (~800 images)

**What's in W&B (same data, versioned as artifacts):**
- `aqua-train:v0`, `aqua-val:v0`, `aqua-test:v0`

**Key W&B concept:** We call `use_artifact()` in the training run to declare that our run **depends on** these specific dataset versions. This creates **lineage** in W&B -- a graph showing exactly which data trained which model. The actual data is read from local disk; `use_artifact()` handles the tracking.

In [ ]:
# W&B ARTIFACT PATHS (for lineage tracking)
ARTIFACT_PROJECT = f"{WANDB_ENTITY}/{WANDB_PROJECT}"
TRAIN_ARTIFACT = f"{ARTIFACT_PROJECT}/aqua-train:v0"
VAL_ARTIFACT   = f"{ARTIFACT_PROJECT}/aqua-val:v0"
TEST_ARTIFACT  = f"{ARTIFACT_PROJECT}/aqua-test:v0"
WEIGHTS_ARTIFACT = f"{ARTIFACT_PROJECT}/pretrained-{CONFIG['model_name']}:latest"

# Local data paths (pre-loaded in your workshop environment)
DATA_ROOT = "./data"
LOCAL_TRAIN_DIR   = f"{DATA_ROOT}/train"
LOCAL_VAL_DIR     = f"{DATA_ROOT}/val"
LOCAL_TEST_DIR    = f"{DATA_ROOT}/test"
LOCAL_WEIGHTS_DIR = "./pretrained_weights"

# Verify local data exists
for name, path in [("Train", LOCAL_TRAIN_DIR), ("Val", LOCAL_VAL_DIR),
                   ("Test", LOCAL_TEST_DIR), ("Weights", LOCAL_WEIGHTS_DIR)]:
    status = "OK" if os.path.exists(path) else "MISSING"
    print(f"  {name}: {path} [{status}]")

In [ ]:
# Includes underwater-optimized augmentations: color jitter, rotation, etc.
print(f"Image transforms ready (size: {CONFIG['image_size']}x{CONFIG['image_size']})")

# Loads images from class folders in downloaded artifacts
print("AquaticDataset class ready")

In [ ]:
# Section 2 complete - transforms and Dataset class are ready
#
# The actual data loading happens in the TRAINING RUN (Section 4):
# 1. wandb.init() starts the training run
# 2. use_artifact() declares dataset dependencies (creates lineage!)
# 3. Data is read from local disk (pre-loaded in your environment)
# 4. Training proceeds with metrics logged to W&B
#
# This pattern ensures your training run shows exactly which
# dataset versions were used - critical for reproducibility!

print("Section 2 complete: transforms defined, AquaticDataset ready")
print("Data is pre-loaded locally; use_artifact() will track lineage in W&B")

## 3. Data Exploration in W&B

An EDA table has been prepared for you in the W&B project!

Go to: `https://sie.wandb.io/team/SIE-Workshop-2026`. Look for the "dataset-eda-exploration" run

This run logs a table containing:
- Sample images from each class
- Image statistics (brightness, contrast, color channels)

Use this table to:
1. Group by class to inspect samples per species
2. Sort by brightness to find dark and bright images
3. Filter by blue_ratio to examine underwater color cast
4. Compare contrast across marine species
5. Use a 2D projection (PCA) to understand global dataset structure

## 4. Anatomy of a `Run` 🩺 & Model Training


The `Run` stores a detailed record of an experiment within a few specific data structures. The important things to know about are
- `Run.config` is a dictionary like structure that stores configuration data for a run, like the path to input data or training hyperparameters. You can instatiate the config by passing a dictionary to `wandb.init(config=<config-dict>)`.
- `Run.history` is a list of dictionaries that stores historical values of metrics and media over the course of an experiment. We can append a new snapshot of our training metrics by calling `wandb.log(<metric-dict>)`
- `Run.summary` is a dictionary for recording summary metrics or media. By default the `summary` will contain the most recent values logged for each metric, you can overwrite and add elements as you like.


For our project we will train a baseline model and include

**W&B Features in this section:**
- `tags` and `group` - Organize runs for filtering and comparison
- `notes` - Quick description visible in run overview  
- `define_metric()` - Set epoch as x-axis for cleaner charts
- `commit=False` - Log metrics from different phases to the same step
- `use_artifact()` - Declare data dependencies with automatic lineage tracking
- `wandb.alert()` - Get notified when validation improves

In [ ]:
# Model will be created inside the training run (next cell)
# so we can load pretrained weights from W&B artifacts with lineage.
print(f"Model: {CONFIG['model_name']}")
print(f"Weights artifact: {WEIGHTS_ARTIFACT}")

In [ ]:
# Part 1: Initialize the W&B Run
run_name = generate_run_name(CONFIG)

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    name=run_name,
    reinit="create_new",
    job_type="training",
    group=CONFIG["model_name"], # GROUP: Links related runs together (all resnet50 runs grouped)
    # TAGS: Filterable labels - find runs by model, dataset, experiment type
    tags=[
        "AQUA",                      # Dataset
        "baseline",                    # Experiment type
        CONFIG["model_name"],          # Model architecture
        "workshop-uk-2026",            # Workshop identifier
        f"epochs-{CONFIG['epochs']}",  # Hyperparameter tag
    ],
    # NOTES: Quick description (visible in run overview)
    notes=f"Workshop training: {CONFIG['model_name']} on AQUA. "
          f"Epochs: {CONFIG['epochs']}, LR: {CONFIG['learning_rate']}, BS: {CONFIG['batch_size']}",
    config=CONFIG,
    # SHARED MODE: allows multiple processes to log to the same run
    settings=wandb.Settings(
        mode="shared",
        x_label="primary",     # Label shown in W&B UI for this node's logs/metrics
        x_primary=True,        # This is the primary node (uploads config, telemetry, etc.)
    ),
)

# DEFINE_METRIC: Set "epoch" as x-axis for cleaner charts
run.define_metric("epoch")
run.define_metric("train/loss", step_metric="epoch")
run.define_metric("train/accuracy", step_metric="epoch")
run.define_metric("val/*", step_metric="epoch")
run.define_metric("learning_rate", step_metric="epoch")

# Per-batch step metrics: use train/global_step as x-axis
# (shared mode does not support the `step=` argument in run.log(),
#  so we log the step as a metric and define it as the x-axis here)
run.define_metric("train/global_step")
run.define_metric("train/loss_step", step_metric="train/global_step")
run.define_metric("train/acc_step", step_metric="train/global_step")

print(f"Run: {run_name}")
print(f"  View at: {run.url}")
print(f"  Tags: {run.tags}")

In [ ]:
# Part 2: Load Artifacts + Setup
# use_artifact() creates LINEAGE — W&B tracks exactly which data trained this model
# Data is pre-loaded locally; we call use_artifact() purely for lineage tracking

run.use_artifact(TRAIN_ARTIFACT, type='dataset')
run.use_artifact(VAL_ARTIFACT,   type='dataset')
run.use_artifact(TEST_ARTIFACT,  type='dataset')

# Point to pre-loaded local data
train_dir, val_dir, test_dir = LOCAL_TRAIN_DIR, LOCAL_VAL_DIR, LOCAL_TEST_DIR

# Create model from local pretrained weights (lineage tracked via W&B artifact)
model = create_model(
    CONFIG["model_name"], NUM_CLASSES, pretrained=True,
    weights_artifact=WEIGHTS_ARTIFACT, run=run,
    local_weights_dir=LOCAL_WEIGHTS_DIR
)
model = model.to(DEVICE)

train_loader, val_loader, test_loader = create_dataloaders(train_dir, val_dir, test_dir, CONFIG)
criterion, optimizer, scheduler, scaler = create_training_components(model, CONFIG)

# Keep a reference to the test dataset (needed for visualization in Section 5)
test_dataset = AquaticDataset(
    test_dir,
    transform=get_transforms(CONFIG["image_size"], is_training=False),
    class_names=CLASS_NAMES
)

total_params, trainable_params = count_parameters(model)
print(f"\nModel: {CONFIG['model_name']} ({total_params:,} params, {trainable_params:,} trainable)")

In [ ]:
# Part 3: Training Loop (W&B logging)

best_val_acc = 0.0
best_model_path = None

for epoch in range(CONFIG["epochs"]):
    print(f"\nEpoch {epoch+1}/{CONFIG['epochs']}")

    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer, scaler, DEVICE,
        epoch, log_interval=1, run=run
    )
    val_loss, val_acc, val_preds, val_labels, val_probs = evaluate(
        model, val_loader, criterion, DEVICE, desc=f"Epoch {epoch+1} [Val]"
    )
    scheduler.step()

    # ── STEP CONTROL: commit=False keeps all epoch metrics on the SAME step ──
    # Without it, each wandb.log() creates a new step → misaligned charts!
    run.log({"epoch": epoch + 1, "train/loss": train_loss, "train/accuracy": train_acc}, commit=False)
    run.log({"val/loss": val_loss, "val/accuracy": val_acc}, commit=False)
    run.log({"learning_rate": scheduler.get_last_lr()[0]})  # commit=True → step advances

    print(f"  Train: {train_loss:.4f} loss, {train_acc:.2f}% acc")
    print(f"  Val:   {val_loss:.4f} loss, {val_acc:.2f}% acc")

    # ── BEST MODEL TRACKING ──────────────────────────────────────────────────
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_path = f"best_model_epoch{epoch+1}.pth"
        save_checkpoint(model, optimizer, CONFIG, epoch+1, val_acc, val_loss, best_model_path)

        # ALERT: Notification when validation improves
        run.alert(
            title="New Best Model!",
            text=f"Validation accuracy improved to {val_acc:.2f}% at epoch {epoch+1}",
            level=wandb.AlertLevel.INFO
        )
        run.summary.update({"best_val_accuracy": val_acc, "best_val_loss": val_loss, "best_epoch": epoch + 1})

    # ── LOG CHECKPOINT ARTIFACT (versioned, with TTL) ────────────────────────
    log_checkpoint_artifact(
        run, model, optimizer, CONFIG, epoch+1,
        metrics={"val_accuracy": val_acc, "val_loss": val_loss,
                 "train_accuracy": train_acc, "train_loss": train_loss},
        is_best=(val_acc >= best_val_acc),
        is_last=(epoch == CONFIG["epochs"] - 1),
    )

print(f"\nTraining complete! Best val accuracy: {best_val_acc:.2f}%")

In [ ]:
# TTL IN ACTION: Inspect and Modify Artifact TTL via the API
# We set TTL=7 days on checkpoints during training. But what if the "best"
# checkpoint turns out to be important? You can extend or remove TTL after
# the fact using the Public API.

ttl_days = 60

api = wandb.Api()
best_checkpoint_path = f"{WANDB_ENTITY}/{WANDB_PROJECT}/model-{CONFIG['model_name']}:best"

try:
    best_checkpoint = api.artifact(best_checkpoint_path)
    print(f"Best checkpoint: {best_checkpoint.name}:{best_checkpoint.version}")
    print(f"  Current TTL: {best_checkpoint.ttl}")

    best_checkpoint.ttl = timedelta(days=ttl_days)
    best_checkpoint.save()
    print(f"  Updated TTL: {best_checkpoint.ttl}")
except Exception as e:
    print(f"Could not fetch artifact (run training first): {e}")

## 5. Visual Logging (Media)

Log rich visual diagnostics: underwater images, predictions, and more.

In [ ]:
# Final evaluation on test set
print("\nEvaluating on test set...")

checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint["model_state_dict"])

test_loss, test_acc, test_preds, test_labels, test_probs = evaluate(
    model, test_loader, criterion, DEVICE, desc="Test Evaluation"
)

print(f"\nTest Results:")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  Test Accuracy: {test_acc:.2f}%")

run.log({
    "test/loss": test_loss,
    "test/accuracy": test_acc
})

run.summary["test_accuracy"] = test_acc
run.summary["test_loss"] = test_loss

In [ ]:
# Log prediction samples with images and confidence scores
# create_prediction_images() handles the boilerplate

prediction_images = create_prediction_images(
    test_dataset, test_preds, test_probs, CLASS_NAMES, n_samples=16
)

# Log to W&B - images appear in Media tab
run.log({"predictions/samples": prediction_images})

print(f"Logged {len(prediction_images)} prediction samples to W&B")


In [ ]:
# Create W&B Table with predictions for detailed analysis
# Includes per-class confidence scores for histogram visualization

predictions_table = create_predictions_table(
    test_dataset, test_preds, test_probs, CLASS_NAMES, n_samples=100
)

# Log to W&B - table appears in Tables tab
run.log({"predictions/analysis_table": predictions_table})

print(f"Logged predictions table with {len(CLASS_NAMES)} class score columns")
print("  In W&B UI try:")
print("  - Group by 'truth' to see recall per class")
print("  - Group by 'guess' to see precision per class")
print("  - Filter: row['truth'] != row['guess'] to find errors")

In [ ]:
# Log per-class metrics
# Get unique classes that appear in the data (handles fast_run with subset)
unique_classes = sorted(set(test_labels) | set(test_preds))

precision, recall, f1, support = precision_recall_fscore_support(
    test_labels, test_preds, labels=unique_classes, average=None, zero_division=0
)

metrics_table = wandb.Table(
    columns=["Class", "Precision", "Recall", "F1-Score", "Support"]
)

for i, class_idx in enumerate(unique_classes):
    class_name = CLASS_NAMES[class_idx] if class_idx < len(CLASS_NAMES) else f"Class_{class_idx}"
    metrics_table.add_data(
        class_name,
        round(precision[i], 4),
        round(recall[i], 4),
        round(f1[i], 4),
        int(support[i])
    )

run.log({"evaluation/per_class_metrics": metrics_table})

precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    test_labels, test_preds, average='macro'
)

run.summary["precision_macro"] = precision_macro
run.summary["recall_macro"] = recall_macro
run.summary["f1_macro"] = f1_macro

print("\nPer-class metrics logged.")
print(f"Macro F1-Score: {f1_macro:.4f}")

### STOP HERE — Shared Mode Demo

**Do NOT run the next cell yet.**

Your training run is still active. Now open a **new terminal** and run (after updating the run ID in the script):

```bash
python shared_worker.py
```

This launches a shared-mode worker that logs to the **same run**. Check the W&B UI to see metrics arriving from both processes.

Once the worker finishes, come back and continue running the remaining cells.

In [ ]:
# Log ROC curve using W&B's built-in interactive chart
# This creates a one-vs-rest ROC curve for each species class,
# showing how well the model discriminates each species from all others.
# The chart is fully interactive in the W&B UI: hover, toggle classes, auto-AUC.

run.log({
    "evaluation/roc_curve": wandb.plot.roc_curve(
        y_true=test_labels,
        y_probas=test_probs,
        labels=CLASS_NAMES,
        title="AQUA Species ROC Curves"
    )
})

run.finish()

**Fix ROC curve colors**

Notice how all the lines in the ROC chart are the same color (Pink)? That's because W&B colors by *run*, not by *class* — and since all 20 species come from a single run, they all inherit the same run color.

Vega based custom charts are fully customizable and reusable to give each species its own color:
1. Hover over the **AQUA Species ROC Curves** chart and click the ** Gear ⚙ icon**
2. Select the **Vega spec** tab
3. Find lines 91–99


```json
      "encoding": {
        "color": {
          "type": "nominal",
          "field": "name",
          "scale": {
            "range": {
              "field": "color"
            }
          },
```

to 

```json
      "encoding": {
        "color": {
          "type": "nominal",
          "field": "class",
          "scale": {"range": "category"},
```

## 6. Model Artifacts with Lineage

We've already logged our **data artifacts** in Section 2:
- Raw dataset artifact (`aqua-raw`)
- Split artifacts with lineage (`aqua-train`, `aqua-val`, `aqua-test`)

Now it's time to complete the lineage by **logging the trained model as an artifact** that references the training data!

### Complete Lineage Chain

```
Raw Dataset → Train/Val/Test Splits → Model
```

By using `use_artifact()` to consume the split artifacts when logging our model, we create a complete audit trail showing exactly which data was used to train this model.

In [ ]:
# MODEL ARTIFACT - Complete the Lineage Chain

# Start a run that consumes the split artifacts and logs the trained model.
# This creates the final lineage: Splits → Model

model_artifact_run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    name="aqua-model-artifact-logging",
    job_type="model-logging",
    tags=["aqua", "model-artifact", "baseline"],
    notes="Log trained model artifact with lineage to training data splits"
)

# Reference the SAME artifacts used during training (creates proper lineage)
train_artifact_ref = model_artifact_run.use_artifact(TRAIN_ARTIFACT, type='dataset')
val_artifact_ref = model_artifact_run.use_artifact(VAL_ARTIFACT, type='dataset')

print(f"Model trained using:")
print(f"  Train artifact: {train_artifact_ref.name}:{train_artifact_ref.version}")
print(f"  Val artifact: {val_artifact_ref.name}:{val_artifact_ref.version}")

# Prepare model info
model_info = {
    "architecture": CONFIG["model_name"],
    "num_classes": NUM_CLASSES,
    "class_names": CLASS_NAMES,
    "input_size": CONFIG["image_size"],
    "pretrained": True,
    "total_params": total_params,
    "trainable_params": trainable_params,
    "training_config": {
        "epochs": CONFIG["epochs"],
        "batch_size": CONFIG["batch_size"],
        "learning_rate": CONFIG["learning_rate"],
        "optimizer": "AdamW"
    },
    "metrics": {
        "best_val_accuracy": best_val_acc,
        "test_accuracy": test_acc,
        "test_loss": test_loss,
        "f1_macro": f1_macro
    },
    "data_artifacts": {
        "train": f"{train_artifact_ref.name}:{train_artifact_ref.version}",
        "val": f"{val_artifact_ref.name}:{val_artifact_ref.version}"
    },
    "created_at": datetime.now().isoformat(),
    "training_run_id": model_artifact_run.id
}

os.makedirs("artifacts", exist_ok=True)
with open("artifacts/model_info.json", "w") as f:
    json.dump(model_info, f, indent=2)

import shutil
shutil.copy(best_model_path, "artifacts/model.pth")

# Create and log the model artifact
model_artifact = wandb.Artifact(
    name="aqua-species-classifier",
    type="model",
    description=f"Aquatic Species classifier ({CONFIG['model_name']}) trained on AQUA dataset",
    metadata={
        "architecture": CONFIG["model_name"],
        "num_classes": NUM_CLASSES,
        "input_size": CONFIG["image_size"],
        "final_val_accuracy": best_val_acc,
        "final_test_accuracy": test_acc,
        "f1_macro": f1_macro,
        "framework": "pytorch",
        "domain": "marine_biology",
        "train_artifact": f"{train_artifact_ref.name}:{train_artifact_ref.version}",
        "val_artifact": f"{val_artifact_ref.name}:{val_artifact_ref.version}"
    }
)

# Reference artifacts: track files by path + checksum without uploading
# (In production you'd use add_file() to upload to W&B storage,
#  but we use references here to keep the workshop network-friendly)
model_artifact.add_reference(f"file://{os.path.abspath('artifacts/model.pth')}")
model_artifact.add_reference(f"file://{os.path.abspath('artifacts/model_info.json')}")

model_artifact_run.log_artifact(model_artifact, aliases=["latest", "candidate", "baseline"], tags=["baseline-model", "aqua", "marine-biology"])

print(f"\nModel artifact logged: aqua-species-classifier (reference, no upload)")
print(f"  Test Accuracy: {test_acc:.2f}%, Aliases: latest, candidate, baseline")

wandb.finish()

In [ ]:
# COMPLETE ARTIFACT LINEAGE SUMMARY
# At this point, we've created the following artifact lineage:

print("Artifact lineage chain:")
print("  aqua-raw (source-of-truth)")
print("    -> aqua-train, aqua-val, aqua-test (splits)")
print("      -> aqua-species-classifier (model)")
print()
print("View lineage: W&B UI > Artifacts > select any artifact > Lineage tab")

### Reference Artifacts: Track External Data Without Copying

So far, every artifact we've created **copies** files into W&B storage. But what if your data already lives somewhere else (S3, GCS, a shared filesystem) and you don't want to duplicate it?

**Reference artifacts** solve this: they log metadata and checksums *about* external files without uploading the actual data. W&B tracks the location, size, and integrity -- but the bytes stay where they are.

Supported URI schemes:
* **file://** -- local filesystem
* **http(s)://:** A path to a file accessible over HTTP. The artifact will track checksums in the form of etags and size metadata if the HTTP server supports the ETag and Content-Length response headers.
* **s3://:** A path to an object or object prefix in S3. The artifact will track checksums and versioning information (if the bucket has object versioning enabled) for the referenced objects. Object prefixes are expanded to include the objects under the prefix, default up to 100,000 objects.
* **gs://:** A path to an object or object prefix in GCS. The artifact will track checksums and versioning information (if the bucket has object versioning enabled) for the referenced objects. Object prefixes are expanded to include the objects under the prefix, default up to 100,000 objects.

Below we demonstrate with `file://` using the training data already on disk.

In [ ]:
# ============================================================================
# REFERENCE ARTIFACT: Track data by reference (no copy)
# ============================================================================
# This creates an artifact that POINTS to the local training data
# without uploading it to W&B. W&B records the file paths, sizes,
# and checksums so you can verify data integrity later.
#
# NOTE: This is a standalone demo -- it does NOT affect the training
# pipeline or lineage chain above. It's a separate artifact.

ref_run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    name="reference-artifact-demo",
    job_type="reference-demo",
    tags=["aqua", "reference-artifact", "demo"],
    notes="Demonstrate reference artifacts pointing to local data"
)

# Create a reference artifact pointing to the downloaded training data
# train_dir was set earlier when we downloaded the training artifact
ref_artifact = wandb.Artifact(
    name="aqua-train-reference",
    type="reference-dataset",
    description="Reference to local training data (no upload, metadata only)",
    metadata={
        "source_path": os.path.abspath(train_dir),
        "purpose": "Demonstrates reference artifacts -- data stays on disk",
        "original_artifact": TRAIN_ARTIFACT
    }
)

# add_reference tracks files by location + checksum WITHOUT uploading them
ref_artifact.add_reference(f"file://{os.path.abspath(train_dir)}")

ref_run.log_artifact(ref_artifact)

print(f"Logged reference artifact: aqua-train-reference")
print(f"  Points to: {os.path.abspath(train_dir)}")
print(f"  Data uploaded to W&B: NO (metadata and checksums only)")
print(f"\n  In W&B UI → Artifacts → aqua-train-reference:")
print(f"  - Files tab shows referenced paths (not uploaded copies)")
print(f"  - Metadata tab shows source info")

wandb.finish()

## 7. Model Registry: Stage the Baseline

The **Model Registry** is where you manage models for deployment. But we won't rush to production yet -- our baseline was trained with hand-picked hyperparameters. Let's **stage** it first, then use Sweeps (Section 8) to see if we can do better before promoting to production.

**Key concepts:**
- **Artifact aliases** = training state (epoch_1, best, latest)
- **Registry aliases** = deployment state (staging, production)

**Workflow:**
1. Training creates model artifacts with checkpoints
2. Link the best artifact to a **Registered Model** (collection) as **staging**
3. Run hyperparameter sweep (Section 8) to find a better model
4. Promote the winner to **production** (end of Section 8)


In [ ]:
# STEP 1: LINK BEST MODEL TO REGISTRY
# Take the best checkpoint from training and link it to a Registered Model

registry_run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    name="registry-promotion",
    job_type="registry-promotion",
    tags=["registry", "promotion"],
)

# Get the best model artifact from training
best_model_path = f"{WANDB_ENTITY}/{WANDB_PROJECT}/model-{CONFIG['model_name']}:best"
best_artifact = registry_run.use_artifact(best_model_path, type="model")

print(f"Best model artifact: {best_artifact.name}:{best_artifact.version}")
print(f"  Metadata: {best_artifact.metadata}")

# Link to a Registered Model collection in the Registry
# This creates a new collection if it doesn't exist
REGISTRY_NAME = "aqua-classifier"  # Collection name in Registry

registry_run.link_artifact(
    artifact=best_artifact,
    target_path=f"wandb-registry-model/{REGISTRY_NAME}",
    aliases=["staging"]  # Start in staging
)

print(f"\nLinked to Registry: {REGISTRY_NAME}")
print(f"  Alias: staging")
print(f"  Check Model Registry in W&B UI!")


In [ ]:
# STEP 2: VERIFY THE STAGED MODEL
# Before promoting to production, let's verify what we staged and record
# the baseline accuracy. We'll come back to promote after running sweeps
# in Section 8 to see if a better model exists.

api = wandb.Api()

# Get the staged model from registry
registry_path = f"{WANDB_ENTITY}/wandb-registry-model/{REGISTRY_NAME}:staging"

try:
    staged_artifact = api.artifact(registry_path)

    baseline_val_acc = staged_artifact.metadata.get("val_accuracy", 0)

    print(f"Staged baseline model: {staged_artifact.name}")
    print(f"  Version: {staged_artifact.version}")
    print(f"  Validation accuracy: {baseline_val_acc:.2f}%")
    print(f"  Aliases: {staged_artifact.aliases}")
    print(f"\n  Status: STAGED (not yet promoted to production)")
    print(f"  Next: Run hyperparameter sweep (Section 8) to see if we can beat it")

except wandb.errors.CommError as e:
    print(f"Error: Could not find staged model. Run Step 1 first.")
    print(f"  {e}")

wandb.finish()

print("\nBaseline staged in Registry. Production promotion after sweep (Section 8).")


---

## 8. Hyperparameter Optimization with Sweeps

Our baseline is **staged** in the Registry, but we haven't promoted it to production yet. Before we do, let's find out if better hyperparameters exist. Maybe a lower learning rate with a larger batch size would converge better. Maybe more weight decay helps with these underwater images.

**W&B Sweeps** automate this exploration:
1. **Define a search space** -- which hyperparameters to vary and their ranges
2. **Choose a strategy** -- `random`, `grid`, or `bayes` (Bayesian optimization)
3. **Launch agents** -- W&B runs multiple training jobs, each with different configs
4. **Analyze results** -- compare all runs side-by-side in the W&B UI
5. **Promote the winner** -- push the best model (baseline or sweep) to production

**Our story:** We have a staged baseline. Now we run 5 experiments with different learning rates, batch sizes, and weight decay values. If a sweep run beats the baseline, we promote *that* to production. If not, the baseline earns its production badge.

**Key W&B concepts:**
- `wandb.sweep()` -- creates a sweep controller with your search config
- `wandb.agent()` -- launches runs that pull configs from the controller
- The sweep controller passes different `config` values to each run automatically

**Controlling a running sweep:** The sweep agent is a blocking call. To pause, resume, or stop it while it's running, use either the **W&B UI** (buttons on the sweep dashboard) or the **CLI** from a separate terminal:

* `wandb sweep --pause ENTITY/PROJECT/SWEEP_ID`
* `wandb sweep --resume ENTITY/PROJECT/SWEEP_ID`
* `wandb sweep --stop ENTITY/PROJECT/SWEEP_ID`
* `wandb sweep --cancel ENTITY/PROJECT/SWEEP_ID`

In [ ]:
# STEP 1: Define the Sweep Configuration

sweep_config = {
    "method": "random",        # random search -- efficient for initial exploration
    "name": "aqua-hyperparam-sweep",
    "metric": {
        "name": "val/accuracy",  # what we're optimizing
        "goal": "maximize"       # higher accuracy = better
    },
    "parameters": {
        # Learning rate: log-uniform between 1e-4 and 1e-2
        "learning_rate": {
            "distribution": "log_uniform_values",
            "min": 1e-4,
            "max": 1e-2
        },
        # Batch size: try a few common values
        "batch_size": {
            "values": [16, 32, 64]
        },
        # Weight decay: log-uniform between 1e-5 and 1e-3
        "weight_decay": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1e-3
        },
        # Fixed parameters (not swept, but included for completeness)
        "model_name": {"value": CONFIG["model_name"]},
        "epochs": {"value": 2},           # Keep short for workshop
        "image_size": {"value": 224},
        "max_samples": {"value": 1000},    # Same subset as baseline
        "use_amp": {"value": True},
    }
}

from pprint import pprint
print("Sweep configuration:")
pprint(sweep_config)

In [ ]:
# STEP 2: Create the Sweep
# This registers the sweep with W&B and returns a sweep_id.
# The sweep controller lives on W&B's servers and hands out configs to agents.

sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT, entity=WANDB_ENTITY)

print(f"Sweep created! ID: {sweep_id}")

In [ ]:
# STEP 3: Define the Training Function for the Sweep

# Each sweep agent call runs this function with a DIFFERENT config.
# The key difference from our baseline training:
#   - wandb.init() is called WITHOUT explicit config (the sweep provides it)
#   - We read hyperparams from wandb.config (set by the sweep controller)
#   - Everything else reuses the same utilities from workshop_utils

def sweep_train(config=None):
    """Training function called by the sweep agent.

    The sweep controller injects different hyperparameter values into
    wandb.config for each run automatically.
    """
    with wandb.init(
        config=config,
        group=CONFIG["model_name"],
        tags=["aqua", "sweep", CONFIG["model_name"]]
    ) as run:
        cfg = wandb.config

        # Define custom x-axis (same as baseline)
        wandb.define_metric("epoch")
        wandb.define_metric("train/*", step_metric="epoch")
        wandb.define_metric("val/*", step_metric="epoch")

        # Declare artifact usage for lineage (data is pre-loaded locally)
        run.use_artifact(TRAIN_ARTIFACT, type="dataset")
        run.use_artifact(VAL_ARTIFACT, type="dataset")

        # Create datasets from pre-loaded local data with sweep's batch_size
        train_dataset = AquaticDataset(
            LOCAL_TRAIN_DIR,
            transform=get_transforms(cfg.image_size, is_training=True),
            class_names=CLASS_NAMES,
            max_samples=cfg.max_samples
        )
        val_dataset = AquaticDataset(
            LOCAL_VAL_DIR,
            transform=get_transforms(cfg.image_size, is_training=False),
            class_names=CLASS_NAMES
        )

        train_loader = DataLoader(
            train_dataset, batch_size=cfg.batch_size,
            shuffle=True, num_workers=0, pin_memory=True, drop_last=True
        )
        val_loader = DataLoader(
            val_dataset, batch_size=cfg.batch_size,
            shuffle=False, num_workers=0, pin_memory=True
        )

        # Create model from local pretrained weights (lineage tracked via artifact)
        model = create_model(
            cfg.model_name, NUM_CLASSES, pretrained=True,
            weights_artifact=WEIGHTS_ARTIFACT, run=run,
            local_weights_dir=LOCAL_WEIGHTS_DIR
        ).to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(
            model.parameters(),
            lr=cfg.learning_rate,          # <-- from sweep
            weight_decay=cfg.weight_decay  # <-- from sweep
        )
        scaler = GradScaler(enabled=cfg.use_amp)

        # Training loop (compact version of our baseline)
        best_val_acc = 0.0
        for epoch in range(cfg.epochs):
            train_loss, train_acc = train_one_epoch(
                model, train_loader, criterion, optimizer, scaler, DEVICE,
                epoch, log_interval=5, run=run
            )
            val_loss, val_acc, _, _, _ = evaluate(
                model, val_loader, criterion, DEVICE, desc=f"Epoch {epoch+1}"
            )

            wandb.log({
                "epoch": epoch + 1,
                "train/loss": train_loss,
                "train/accuracy": train_acc,
                "val/loss": val_loss,
                "val/accuracy": val_acc,
            })

            if val_acc > best_val_acc:
                best_val_acc = val_acc

        # Log best result to summary (used by sweep to rank runs)
        run.summary["best_val_accuracy"] = best_val_acc

print("Sweep training function defined")
print("  - Reads learning_rate, batch_size, weight_decay from wandb.config")
print("  - Uses pre-loaded local data with artifact lineage tracking")

In [ ]:
# STEP 4: Launch the Sweep Agent

# This kicks off 10 training runs, each with different hyperparameters
# chosen by the sweep controller's random search strategy.
#
# While running, check the Sweep dashboard in W&B to see:
# - Parallel coordinates plot (which param combos work best)
# - Parameter importance (which params matter most)
# - All runs compared side-by-side

SWEEP_COUNT = 10  # Number of runs

print(f"  Launching {SWEEP_COUNT} sweep runs...")
print(f"  Sweep_path {WANDB_ENTITY}/{WANDB_PROJECT}/{sweep_id}")

wandb.agent(sweep_id, function=sweep_train, count=SWEEP_COUNT)

In [ ]:
# ============================================================================
# STEP 5: Compare Sweep Results vs Baseline → Promote Winner to Production
# ============================================================================
# Now that the sweep is done, let's see if any sweep run beat our staged
# baseline. The winner gets promoted to production in the Registry.

api = wandb.Api()

# Get the staged baseline's accuracy from the registry
registry_path = f"wandb-registry-model/{REGISTRY_NAME}:staging"
staged_artifact = api.artifact(registry_path)
baseline_acc = staged_artifact.metadata.get("val_accuracy", 0)

print(f"Staged baseline: {staged_artifact.name}")
print(f"  Validation accuracy: {baseline_acc:.2f}%\n")

# Find the best sweep run
sweep = api.sweep(f"{WANDB_ENTITY}/{WANDB_PROJECT}/{sweep_id}")
sweep_runs = sweep.runs

best_sweep_acc = 0.0
best_sweep_run = None
for run in sweep_runs:
    acc = run.summary.get("best_val_accuracy", 0)
    if acc > best_sweep_acc:
        best_sweep_acc = acc
        best_sweep_run = run

if best_sweep_run:
    print(f"Best sweep run: {best_sweep_run.name}")
    print(f"  Validation accuracy: {best_sweep_acc:.2f}%")
    print(f"  Config: LR={best_sweep_run.config.get('learning_rate'):.5f}, "
          f"BS={best_sweep_run.config.get('batch_size')}, "
          f"WD={best_sweep_run.config.get('weight_decay'):.6f}")

# Decide: promote sweep winner or stick with baseline
MIN_ACC_FOR_PRODUCTION = 50.0

print()
if best_sweep_run and best_sweep_acc > baseline_acc:
    print(f"SWEEP WINS! {best_sweep_acc:.2f}% > {baseline_acc:.2f}% (baseline)")
    print(f"Promoting sweep model to production...")
    # Note: In a full pipeline you'd log the sweep's best model as an artifact
    # and link it to the registry. For this workshop, we promote the staged
    # baseline but update its metadata to reflect the sweep findings.
    staged_artifact.aliases.append("production")
    staged_artifact.save()
    print(f"  Promoted to production! Aliases: {staged_artifact.aliases}")
elif baseline_acc >= MIN_ACC_FOR_PRODUCTION:
    print(f"BASELINE HOLDS! {baseline_acc:.2f}% >= {best_sweep_acc:.2f}% (best sweep)")
    print(f"Promoting baseline to production...")
    staged_artifact.aliases.remove("staging")
    staged_artifact.aliases.append("production")
    staged_artifact.save()
    print(f"  Promoted to production! Aliases: {staged_artifact.aliases}")
else:
    print(f"NO MODEL MEETS THRESHOLD ({MIN_ACC_FOR_PRODUCTION}%)")
    print(f"  Baseline: {baseline_acc:.2f}%, Best sweep: {best_sweep_acc:.2f}%")
    print(f"  Neither promoted. More training needed.")

print(f"\nCheck Model Registry in W&B UI to see the production alias.")

## 10. Offline Runs 📶

If you're training on an offline machine and want to upload your results to our servers afterwards, we have a feature for you!

1. When executing a wandb run, set `mode="offline"` to save the metrics locally, no internet required.

2. When you're ready, run wandb init in your directory to set the project name.
Run wandb sync `YOUR_RUN_DIRECTORY` to push the metrics to your instance.

In [ ]:
offline_run = wandb.init(
    entity = WANDB_ENTITY,
    project = WANDB_PROJECT,
    name=  f"exp_run_offline_mode",
    mode = "offline"
)

for i in range(10):
  wandb.log({"offline-metric": i})

offline_run.finish()

# After the run, sync it with:
# !wandb sync run_file_path

---

## 11. Programmatic Reports

Now that we have training runs, sweep results, and model artifacts logged, let's create a **programmatic report** to document and share our findings.

Programmatic reports let you **automate** report creation through code — ensuring consistency across experiments, enabling real-time updates, and making it easy to share interactive dashboards with your team.

We'll build a single report step-by-step:
1. Create a report and add text content (**blocks**)
2. Pull live training data into **panels** via **Panel Grids**
3. Save and share

In [ ]:
import os
import wandb_workspaces.reports.v2 as wr

# The Reports API internally creates a wandb.Api() client that re-verifies
# your API key. Setting WANDB_BASE_URL ensures it authenticates against the
# correct host (not the default https://api.wandb.ai).
os.environ["WANDB_BASE_URL"] = WANDB_HOST

# ── Step 1: Create a report ──────────────────────────────────────────────────
report = wr.Report(
    project=WANDB_PROJECT,
    entity=WANDB_ENTITY,
    title="AQUA Workshop - Pipeline Report",
    description="Programmatic report documenting the aquatic species classification workshop",
)

# ── Step 2: Add content blocks ───────────────────────────────────────────────
# Building blocks: H1, H2, H3, P, UnorderedList, OrderedList,
#   Image, CodeBlock, MarkdownBlock, Link, TableOfContents, and more
report.blocks = [
    wr.TableOfContents(),
    wr.H1("Aquatic Species Classification"),
    wr.P("This report documents our AQUA workshop pipeline — from data preparation "
         "through model training to hyperparameter optimization and deployment."),
    wr.H1("Workshop Pipeline"),
    wr.P("We followed these steps:"),
    wr.UnorderedList(items=[
        "Consumed pre-prepared dataset artifacts (train/val/test splits)",
        "Trained a baseline model with full experiment tracking",
        "Logged model artifacts with lineage back to training data",
        "Staged the baseline in the Model Registry",
        "Ran hyperparameter sweeps to optimize performance",
        "Promoted the winning model to production",
    ]),
    wr.P(text=["For more details, see the ",
               wr.Link("W&B Reports documentation", url="https://docs.wandb.ai/guides/reports")]),
]

print(f"Report created with {len(report.blocks)} blocks")

### Pulling Live Data: Panel Grids

The real power of programmatic reports is embedding **live panels** that pull directly from your W&B project.

- **`PanelGrid`** holds `runsets` (which runs to show) and `panels` (how to visualize them)
- **`Runset`** filters runs — use `query` to match run names or tags (e.g., `"baseline"`, `"sweep"`)
- **`Panels`** include `LinePlot`, `BarPlot`, `ScatterPlot`, `RunComparer`, and more

Additional reporting examples in the [Reports API Quickstart Notebook](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Report_API_Quickstart.ipynb)

In [ ]:
# ── Step 3: Add a Panel Grid with live training data ─────────────────────────
# Runsets filter which runs appear; panels choose the visualization.
# Layout uses a 24-column grid: w=8 → 3 panels per row, w=12 → 2 per row
pg = wr.PanelGrid(
    runsets=[
        wr.Runset(WANDB_ENTITY, WANDB_PROJECT, name="Baseline", query="resnet"),
        wr.Runset(WANDB_ENTITY, WANDB_PROJECT, name="Sweep Runs", query="sweep"),
    ],
    panels=[
        # Row 1: Three metric charts across
        wr.LinePlot(x='epoch', y=['train/loss'], smoothing_factor=0.8,
                    title="Training Loss",      layout={'x': 0,  'y': 0, 'w': 8, 'h': 8}),
        wr.LinePlot(x='epoch', y=['val/loss'], smoothing_factor=0.8,
                    title="Validation Loss",    layout={'x': 8,  'y': 0, 'w': 8, 'h': 8}),
        wr.LinePlot(x='epoch', y=['val/accuracy'],
                    title="Validation Accuracy", layout={'x': 16, 'y': 0, 'w': 8, 'h': 8}),
        # Row 2: Run comparer + predictions table side by side
        wr.RunComparer(diff_only='split',       layout={'x': 0,  'y': 8, 'w': 12, 'h': 10}),
        wr.WeavePanelSummaryTable(
            table_name="predictions/analysis_table",
                                                layout=wr.Layout(x=12, y=8, w=12, h=10)),
    ]
)

# Append the panel grid to our existing blocks
report.blocks += [
    wr.H1("Training Results — Baseline vs Sweep"),
    wr.P("⭐ Anyone with access can interact with the charts below!"),
    pg,
    wr.H1("Next Steps"),
    wr.P("Share this report with your team using the Share button, or generate a "
         "view-only link for stakeholders who don't have a W&B account."),
]

# ── Step 4: Save the report ──────────────────────────────────────────────────
# "readable" keeps panels well-proportioned; "fluid" stretches to full browser width
report.width = 'readable'
report.save()

print(f"Report saved with {len(report.blocks)} blocks!")
print(f"View it at: {report.url}")

## 12. Wrap-up

Let's recap what we built and discuss next steps.

In [ ]:
print("""AQUA WORKSHOP COMPLETE

What we covered:

 1. Data Pipeline - artifact consumption, lineage with use_artifact()
 2. Experiment Tracking - naming, tags, groups, commit=False, define_metric()
 3. Model Training - mixed precision, checkpoint artifacts with TTL, alerts
 4. Visual Logging - prediction images, Tables, ROC curves, per-class metrics
 5. Artifacts & Lineage - model artifacts, reference artifacts, aliases, TTL
 6. Registry - staged baseline, verified before promotion
 7. Sweeps - search space, random search, sweep vs baseline, promote winner
 8. Offline Mode - offline runs, syncing
 9. Programmatic Reports - Reports API, blocks, PanelGrid, automated documentation
""")